***
## Import

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from datetime import datetime, timedelta # Dùng để xử lý dữ liệu thời gian

import urllib.robotparser # Kiểm tra file robot.txt có được phép crawl không
from time import sleep
from selenium import webdriver
from selenium.webdriver import DesiredCapabilities
import json

## Lấy client_id của trang https://soundcloud.com/ 
Ta sẽ gửi requests đến file Chrome log của trang Soundcloud từ đó trích các request get_log trong file Chrome log.
Thông qua các kết quả đã được trích, ta sẽ tìm được client id.
- Code được tham khảo từ trang: https://gist.github.com/lorey/079c5e178c9c9d3c30ad87df7f70491d#fileselenium_xhr_requests_via_performance_logging-py

In [ ]:
# Yeu cau request den Chrome log
capabilities = DesiredCapabilities.CHROME
capabilities["loggingPrefs"] = {"performance": "ALL"}  # newer: goog:loggingPrefs
capabilities["goog:loggingPrefs"] = {"performance": "ALL"}
driver = webdriver.Chrome(
    desired_capabilities=capabilities, executable_path="chromedriver.exe"
)

# Gui request đến trang Soundcloud
driver.get("https://soundcloud.com")
sleep(5)  # wait for the requests to take place

# Trich request ra tu log
logs_raw = driver.get_log("performance")
logs = [json.loads(lr["message"])["message"] for lr in logs_raw]

def log_filter(log_):
    return (
        # is an actual response
        log_["method"] == "Network.responseReceived"
        # and json
        and "json" in log_["params"]["response"]["mimeType"]
    )

#Tach client_id tu requests nhan duoc
for log in filter(log_filter, logs):
    resp_url = log["params"]["response"]["url"]
    if 'client_id' in resp_url: 
        a = resp_url.split('?')[1]
        b = a.split('&')[0]
        break
client_id = b.split('=')[1]

#Dong chrome
driver.close()

## Các thể loại playlist được tìm : (tổng cộng có 43 thể loại) 

In [ ]:
genres = ['soundcloud:genres:all-music','soundcloud:genres:all-audio','soundcloud:genres:alternativerock',
         'soundcloud:genres:ambient','soundcloud:genres:classical','soundcloud:genres:country',
         'soundcloud:genres:danceedm','soundcloud:genres:dancehall','soundcloud:genres:deephouse',
         'soundcloud:genres:disco','soundcloud:genres:drumbass','soundcloud:genres:dubstep',
         'soundcloud:genres:electronic','soundcloud:genres:folksingersongwriter','soundcloud:genres:hiphoprap',
         'soundcloud:genres:house','soundcloud:genres:indie','soundcloud:genres:jazzblues',
         'soundcloud:genres:latin','soundcloud:genres:metal','soundcloud:genres:piano','soundcloud:genres:pop',
         'soundcloud:genres:rbsoul','soundcloud:genres:reggae','soundcloud:genres:reggaeton',
         'soundcloud:genres:rock','soundcloud:genres:soundtrack','soundcloud:genres:techno',
         'soundcloud:genres:trance','soundcloud:genres:trap','soundcloud:genres:triphop',
         'soundcloud:genres:world','soundcloud:genres:audiobooks','soundcloud:genres:business',
         'soundcloud:genres:comedy','soundcloud:genres:entertainment','soundcloud:genres:learning',
         'soundcloud:genres:newspolitics','soundcloud:genres:religionspirituality','soundcloud:genres:science',
         'soundcloud:genres:sports','soundcloud:genres:storytelling','soundcloud:genres:technology']

## Lưu thông tin của playlist:
- playlist_link: Đường dẫn của playlist
- playlist_title: tên playlist
- playlist_genre: thể loại của playlist
- playlist_likesCount: số lượt thích của playlist
- playlist_repostsCount: số lượt repost của playlist
- playlist_createdDate: ngày tạo playlist
- playlist_duration: thời gian chạy playlist (đơn vị ms)
- playlist_trackCount: số lượng track của playlist
- playlist_Id: ID của playlist
- playlist_username: tên user của playlist
- playlist_userId: ID user của playlist

In [ ]:
playlist_link=[];playlist_title=[];playlist_genre=[];playlist_likesCount=[];playlist_repostsCount=[];playlist_createdDate=[]
playlist_duration=[];playlist_username=[];playlist_trackCount=[];playlist_Id =[];playlist_userId=[]

## Lưu thông tin của user:
- user_link: đường dẫn của user              
- user_name: tên user
- user_createdDate: ngày tạo tài khoản của user
- user_followersCount: số lượt follow của user
- user_playlistCount: số playlist user đã tạo
- user_trackCount: số track user đã tạo
- user_likesCount: số lượt thích của user
- user_followingsCount: số người user follow
- user_Id: ID của user

In [ ]:
user_link=[];user_name=[];user_createdDate=[];user_followersCount=[];user_playlistCount=[]
user_trackCount=[];user_likesCount=[];user_followingsCount=[];user_Id=[]

## Lưu thông tin của track:
- track_link: đường dẫn của track
- track_title: tên track
- track_createdDate: ngày tạo track
- track_genre: thể loại của track
- track_playbackCount: số lượt nghe của track
- track_Id: ID của track
- track_likesCount: số lượt thích của track
- track_repostsCount: số lượt repost của track
- track_userId: ID user của track
- track_downloadCount: số lượt tải của track
- track_playlistlink: đường dẫn playlist mà track thuộc về
- track_duration: thời gian chạy track (đơn vị ms)

In [ ]:
track_link=[];track_title=[];track_createdDate=[];track_genre=[];track_playbackCount=[];track_Id=[]
track_likesCount=[];track_repostsCount=[];track_userId=[];track_downloadCount=[];track_playlistlink=[];track_duration=[]

## Với mỗi thể loại sẽ có một API riêng để request từ client id đã có được ở trên
- Mỗi lần request ta lấy 25 playlist đầu tiên được trả về. Từ kết quả request được, ta chuyển về dạng JSON rồi từ đó lấy thông tin của playlist. 
- Ứng với mỗi playlist ta sẽ lấy được user của playlist cùng các track thuộc về playlist đó.
***
## Lưu ý: 
    - 1 playlist thường chỉ hiện thông tin của từ 2-4 track nên đối với các track liền sau ta lấy id của track rồi từ đó đưa vào đường dẫn api mới để request mới có được thông tin đầy đủ của 5 track đầu mỗi playlist.
    - Tương tự, thông tin các user của track cũng sẽ không đầy đủ so với thông tin user của playlist nên ta lấy id của user rồi từ đó đưa vào đường dẫn api mới để request mới có được thông tin đầy đủ của user của mỗi playlist.
    - Đôi lúc đường dẫn api sẽ không request được nên ta phải kiểm tra và request lại 
***
Cuối cùng ta sẽ được khoảng 1000 playlist, 5000 track và 6000 user. Tuy nhiên trong quá trình thu thập dữ liệu sẽ có sự trùng lặp nên sau khi qua xử lý có thể sẽ thu về con số thấp hơn

In [ ]:
#Xet tung the loai trong 43 the loai
for genre in genres:
    if len(playlist_link) <= 1000:
        #Duong dan api voi the loai va client id tuong ung
        link = f'https://api-v2.soundcloud.com/search/playlists_without_albums?q={genre[18:]}&variant_ids=&facet=genre&user_id=610739-751556-838406-110722&client_id={client_id}&limit=25&offset=0&linked_partitioning=1&app_version=1635494121&app_locale=en'

        r = requests.get(link)
        time.sleep(1)
        #Kiem tra request da thanh cong hay chua
        while(len(r.text) <= 10):
            r = requests.get(link)
        #Dua ket qua ve dang json
        json_data = json.loads(r.text)
        playlists = json_data['collection']
        
        #Xet tung playlist trong 1 ket qua request
        for i in range(len(playlists)):
            playlist_link.append(playlists[i]['permalink_url'])
            playlist_title.append(playlists[i]['title'])
            playlist_genre.append(playlists[i]['genre'])
            playlist_likesCount.append(playlists[i]['likes_count'])
            playlist_repostsCount.append(playlists[i]['reposts_count'])
            playlist_createdDate.append(playlists[i]['created_at'])
            playlist_duration.append(playlists[i]['duration'])
            playlist_trackCount.append(playlists[i]['track_count'])
            playlist_username.append(playlists[i]['user']['username'])
            playlist_Id.append(playlists[i]['id'])
            playlist_userId.append(playlists[i]['user_id'])
                
            #Voi moi playlist ta cung lay thong tin user cua playlist do
            user = playlists[i]['user']
            user_link.append(user['permalink_url'])
            user_name.append(user['username'])
            user_createdDate.append(user['created_at'])
            user_followersCount.append(user['followers_count'])
            user_playlistCount.append(user['playlist_count'])
            user_trackCount.append(user['track_count'])
            user_likesCount.append(user['likes_count'])
            user_followingsCount.append(user['followings_count'])
            user_Id.append(user['id'])
                
            #Voi moi playlist ta se xet 5 track dau cua playlist
            for j in range(len(playlists[i]['tracks'])):
                tracks = playlists[i]['tracks'][j]
                id_user = ''
                if j == 5:
                    break
                #Neu thong tin cua 1 track chi co id thi ta se request den api cua chinh track do
                if len(tracks) == 4 and j <= 4:
                    id_track = tracks['id']
                    link_track = f'https://api-v2.soundcloud.com/tracks/{id_track}?client_id=YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79'
                    t = requests.get(link_track)
                    while(len(t.text) <= 10):
                        t = requests.get(link_track)
                    tracks = json.loads(t.text)
                        
                track_playlistlink.append(playlists[i]['permalink_url'])
                track_link.append(tracks['permalink_url'])
                track_title.append(tracks['title'])
                track_createdDate.append(tracks['created_at'])
                track_genre.append(tracks['genre'])
                track_playbackCount.append(tracks['playback_count'])
                track_Id.append(tracks['id'])
                track_likesCount.append(tracks['likes_count'])
                track_repostsCount.append(tracks['reposts_count'])
                track_userId.append(tracks['user_id'])
                track_downloadCount.append(tracks['download_count'])
                track_duration.append(tracks['duration'])
                    
                #Vi thong tin user cua cac track trong playlist khong day du nen ta se tiep tuc request den api cua user do
                id_user = tracks['user']['id'] 
                link_user = f'https://api-v2.soundcloud.com/users/{id_user}?client_id={client_id}'
                u = requests.get(link_user)
                while(len(u.text) <= 10):
                    u = requests.get(link_user)
                user_info = json.loads(u.text)
                user_link.append(user_info['permalink_url'])
                user_name.append(user_info['username'])
                user_createdDate.append(user_info['created_at'])
                user_followersCount.append(user_info['followers_count'])
                user_playlistCount.append(user_info['playlist_count'])
                user_trackCount.append(user_info['track_count'])
                user_likesCount.append(user_info['likes_count'])
                user_followingsCount.append(user_info['followings_count'])
                user_Id.append(user_info['id'])
    else:
        break

## Chuyển các list về dạng dataframe để dễ xử lý

In [ ]:
df_track = pd.DataFrame(zip(track_link,track_title,track_createdDate,track_genre,track_playbackCount,track_Id,track_likesCount,
                            track_repostsCount,track_userId,track_downloadCount,track_playlistlink,track_duration),
                        columns = ['Track Link','Title','Created Date','Genre','Playback Count','ID', 'Likes Count',
                                   'Reposts Count','User ID','Downdload Count','Playlist Link','Duration'])    
df_user = pd.DataFrame(zip(user_link,user_name,user_createdDate,user_followersCount,user_playlistCount,user_trackCount,
                           user_likesCount,user_followingsCount,user_Id),
                       columns = ['User Link','Name','Created Date','Followers Count','Playlist Count','Track Count',
                                  'Likes Count','Followings Count','User ID']) 
df_playlist = pd.DataFrame(zip(playlist_link,playlist_title,playlist_genre,playlist_likesCount,playlist_repostsCount,
                               playlist_createdDate,playlist_duration,playlist_trackCount,
                               playlist_Id,playlist_username,playlist_userId),
                           columns = ['Playlist Link','Title','Genre','Likes Count','Reposts Count',
                                      'Created Date','Duration','Track Count','ID','User Name','User ID'])

## Loại bỏ các link của track và link của user bị trùng lặp 
- Nếu 2 kết quả trùng nhau ta sẽ giữ lại giá trị được thu thập trước
- Sau khi đã loại bỏ, ta cần đánh số các index lại

In [ ]:
#Loai bo trung lap
df_playlist = df_playlist.drop_duplicates(subset='Playlist Link',keep='first')
df_track = df_track.drop_duplicates(subset='Track Link',keep='first')
df_user = df_user.drop_duplicates(subset='User Link',keep='first')
#Reset index
df_playlist = df_playlist.reset_index(drop=True)
df_track = df_track.reset_index(drop=True)
df_user = df_user.reset_index(drop=True)

## Số lượng các playlist, track và user thu thập được : 

In [ ]:
print(f'So luong playlist: {len(df_playlist)}\nSo luong track: {len(df_track)}\nSo luong user: {len(df_user)}')

## Vì 1 playlist có thể có nhiều track nên với mỗi playlist ta lưu 1 string danh sách các id của track. Khi sử dụng chỉ cần tách các số trong string ra là có một danh sách các track của 1 playlist
    - Với mỗi link playlist, ta sẽ so sánh từng link playlist mà 1 track thuộc về. Nếu 2 link như nhau thì ta sẽ thêm id của track vào chuỗi danh sách track của playlist, các id được ngăn cách bởi dấu ";"
    - Đôi lúc sẽ có 1 vài playlist không có track nào trong kết quả là vì có trường hợp 1 playlist chỉ có từ 1-2 track 
    nhưng sau khi lọc các track trùng lặp thì không còn track nào hợp lệ.

In [ ]:
ids = [] 
for playlist in df_playlist['Playlist Link']:
    list_id = []
    for id,link in zip(df_track['ID'],df_track['Playlist Link']):
        if link == playlist:    #Neu link playlist cua track trung voi link cua playlist dang so sanh thi them vao list
            list_id.append(id)
    list_id_str = ''
    for i in range(len(list_id)):
        list_id_str += str(list_id[i]) # Chuyen list id thanh mot chuoi danh sach track id cua playlist
        if i != len(list_id) - 1:   #Cac id duoc ngan cach boi dau phay tru id cuoi
            list_id_str += ';'
    ids.append(list_id_str)
df_playlist['Tracks'] = ids

## Lưu các thông tin của playlist, user và track vào các file csv để tiện tra cứu

In [ ]:
df_playlist.to_csv('./API/playlist.csv')
df_track.to_csv('./API/track.csv')
df_user.to_csv('./API/user.csv')